# Making a Kepler map = python 3.11, numpy version 1.23.5

In [1]:
import pandas as pd
import os
import keplergl
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt
import importlib.resources
from IPython.display import HTML, display
   
def _get_asset_str(filepath: str) -> bytes:
    pkg, sub = 'keplergl.keplergl', filepath
    return importlib.resources.read_binary(pkg, sub)

In [2]:
path = r"C:\Users\ryani\Desktop\JupyterLab\NY_Citibike_2022"

In [37]:
df = pd.read_csv(os.path.join(path, '02_Prepared_Data', 'sampled_top_trips.csv'), index_col=False)
df = df.drop(columns=['Unnamed: 0'])
df

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,date,avgtemp,trips_per_day,tripduration(mins),usertype,month,trip_count
0,Norfolk St & Broome St,Henry St & Grand St,40.717228,-73.988020,40.714210,-73.981094,2022-01-20,5.1,33928,3,member_classic_bike,1,4883
1,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,40.799484,-73.955610,2022-01-16,-8.0,20930,4,casual_classic_bike,1,4260
2,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,40.799484,-73.955610,2022-01-16,-8.0,20930,54,casual_classic_bike,1,4260
3,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,40.799484,-73.955610,2022-01-16,-8.0,20930,55,casual_classic_bike,1,4260
4,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,40.799484,-73.955610,2022-01-16,-8.0,20930,5,casual_electric_bike,1,4260
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167935,Centre St & Chambers St,Centre St & Chambers St,40.712658,-74.004715,40.712734,-74.004610,2022-12-28,3.4,44027,23,casual_electric_bike,12,4051
167936,Centre St & Chambers St,Centre St & Chambers St,40.712734,-74.004610,40.712734,-74.004610,2022-12-27,-0.7,33044,1,member_classic_bike,12,4051
167937,Centre St & Chambers St,Centre St & Chambers St,40.712734,-74.004610,40.712734,-74.004610,2022-12-26,-4.7,19221,4,casual_classic_bike,12,4051
167938,Centre St & Chambers St,Centre St & Chambers St,40.712734,-74.004610,40.712734,-74.004610,2022-12-03,10.6,45429,1,member_electric_bike,12,4051


In [38]:
df.describe()

,start_lat,start_lng,end_lat,end_lng,avgtemp,trips_per_day,tripduration(mins),month,trip_count
count,167940.000000,167940.000000,167940.000000,167940.000000,167940.000000,167940.000000,167940.000000,167940.000000,167940.000000
mean,40.741673,-73.985109,40.741720,-73.985604,18.334157,98198.008616,27.022651,6.899053,5951.477254
std,0.032533,0.021237,0.032875,0.021404,8.106827,25832.471548,72.545894,2.678498,2145.455892
min,40.661037,-74.016780,40.661064,-74.016754,-11.700000,2848.000000,1.000000,1.000000,4051.000000
25%,40.717453,-74.004610,40.714210,-74.004610,13.100000,83439.000000,4.000000,5.000000,4328.000000
50%,40.757282,-73.981094,40.757282,-73.981094,20.200000,106089.000000,12.000000,7.000000,5262.000000
75%,40.765910,-73.973720,40.765910,-73.973720,24.700000,117444.000000,38.000000,9.000000,7275.000000
max,40.811348,-73.915955,40.799484,-73.947720,31.300000,135072.000000,9125.000000,12.000000,12041.000000


In [39]:
df.shape

(167940, 13)

In [40]:
df.columns

Index(['start_station_name', 'end_station_name', 'start_lat', 'start_lng',
       'end_lat', 'end_lng', 'date', 'avgtemp', 'trips_per_day',
       'tripduration(mins)', 'usertype', 'month', 'trip_count'],
      dtype='object')

In [41]:
df_map = df[['start_station_name', 'start_lat', 'start_lng', 'end_station_name', 'end_lat', 'end_lng', 'trip_count']]
df_map.head()

,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,trip_count
0,Norfolk St & Broome St,40.717228,-73.98802,Henry St & Grand St,40.714210,-73.981094,4883
1,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.95561,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,4260
2,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.95561,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,4260
3,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.95561,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,4260
4,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.95561,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,4260


In [42]:
df_map_drop = df_map.copy()
df_map_drop.drop_duplicates(inplace=True)

In [43]:
df_map_drop.dropna(inplace=True)

In [44]:
df_map_drop.shape

(10601, 7)

In [45]:
df_map_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10601 entries, 0 to 167939
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_station_name  10601 non-null  object 
 1   start_lat           10601 non-null  float64
 2   start_lng           10601 non-null  float64
 3   end_station_name    10601 non-null  object 
 4   end_lat             10601 non-null  float64
 5   end_lng             10601 non-null  float64
 6   trip_count          10601 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 662.6+ KB


In [46]:
df_map_drop.describe()

,start_lat,start_lng,end_lat,end_lng,trip_count
count,10601.000000,10601.000000,10601.000000,10601.000000,10601.000000
mean,40.748674,-73.980800,40.748692,-73.981419,5876.647675
std,0.028185,0.016358,0.028539,0.016523,1968.648785
min,40.661037,-74.016780,40.661064,-74.016754,4051.000000
25%,40.741690,-73.987260,40.744750,-73.986580,4556.000000
50%,40.755814,-73.981090,40.755135,-73.981094,5316.000000
75%,40.766018,-73.973820,40.765910,-73.973720,6345.000000
max,40.811348,-73.915955,40.799484,-73.947720,12041.000000


In [47]:
df_map_drop['trip_count'] = pd.to_numeric(df_map_drop['trip_count'], errors='coerce').astype('Int32')
df_map_drop = df_map_drop.dropna(subset=['trip_count'])

In [51]:
df_map_drop

,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,trip_count
0,Norfolk St & Broome St,40.717228,-73.988020,Henry St & Grand St,40.714210,-73.981094,4883
1,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955610,4260
40,Centre St & Chambers St,40.712734,-74.004610,Centre St & Chambers St,40.712734,-74.004610,4051
238,Henry St & Grand St,40.714210,-73.981094,Norfolk St & Broome St,40.717228,-73.988020,4324
469,12 Ave & W 40 St,40.760876,-74.002780,12 Ave & W 40 St,40.760876,-74.002780,5409
...,...,...,...,...,...,...,...
167898,Centre St & Chambers St,40.712814,-74.004610,Centre St & Chambers St,40.712734,-74.004610,4051
167902,Centre St & Chambers St,40.712630,-74.004700,Centre St & Chambers St,40.712734,-74.004610,4051
167920,Centre St & Chambers St,40.712635,-74.004670,Centre St & Chambers St,40.712734,-74.004610,4051
167928,Centre St & Chambers St,40.712677,-74.004690,Centre St & Chambers St,40.712734,-74.004610,4051


In [52]:
map_1 = KeplerGl(height=600)
map_1.add_data(data=df_map_drop, name='Citibike NY 2022')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Citibike NY 2022': {'index': [0, 1, 40, 238, 469, 472, 688, 752, 819, 1115, 1236, 1356, 1430, …

## I changed the colour points to be green at start and red at end, this felt appropriate and colours match nicely. I reduced the size of the lines so that congregated lines show up better. The Arc begins light green and ends pinkinsh, following the colour style. Point size reduced as well.

## A large amount of people are cycling to or down Empire state trail - perhaps exercising? A large amoutn of trips end at 10th Ave and W 26th Street. This area has art galleries, a park, and a night club. A large amount of trips are to or around central park - again perhaps recreation. There are also a large amount of trips from residential areas south of Central Park to the nearby Theatre Area at 10th Ave and 52/53rd street.

In [53]:
config = map_1.config

In [54]:
config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'abbf5y',
     'type': 'point',
     'config': {'dataId': 'Citibike NY 2022',
      'label': 'start',
      'color': [76, 154, 78],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'start_lat', 'lng': 'start_lng'},
      'isVisible': True,
      'visConfig': {'radius': 5,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRan

In [55]:
map_1.save_to_html(file_name = 'NY_Citibike_2022_top.html', read_only = False, config = config)

Map saved to NY_Citibike_2022_top.html!
